In [ ]:
google_colab = False
if google_colab:
    from google.colab import drive
    drive.mount('/content/drive')
    cwd="/content/drive/MyDrive/NewsTrading/trading_bot"
    %cd /content/drive/MyDrive/NewsTrading/trading_bot
    %pip install -r requirements_clean.txt
    !git clone https://github.com/rapidsai/rapidsai-csp-utils.git
    !python rapidsai-csp-utils/colab/pip-install.py

In [22]:
%load_ext autoreload
%autoreload 2
import pandas as pd
import torch
import torch.nn as nn
from torch.optim import AdamW
from transformers import BertTokenizerFast, get_linear_schedule_with_warmup
from src.model.data_loading import get_data_loader_from_dataset
from src.config import config, MODEL_CONFIG
from src.model.neural_network import train, MyBertModule

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


[autoreload of src.model.neural_network failed: Traceback (most recent call last):
  File "g:\Meine Ablage\NewsTrading\trading_bot\.venv\Lib\site-packages\IPython\extensions\autoreload.py", line 276, in check
    superreload(m, reload, self.old_objects)
  File "g:\Meine Ablage\NewsTrading\trading_bot\.venv\Lib\site-packages\IPython\extensions\autoreload.py", line 500, in superreload
    update_generic(old_obj, new_obj)
  File "g:\Meine Ablage\NewsTrading\trading_bot\.venv\Lib\site-packages\IPython\extensions\autoreload.py", line 397, in update_generic
    update(a, b)
  File "g:\Meine Ablage\NewsTrading\trading_bot\.venv\Lib\site-packages\IPython\extensions\autoreload.py", line 349, in update_class
    if update_generic(old_obj, new_obj):
       ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "g:\Meine Ablage\NewsTrading\trading_bot\.venv\Lib\site-packages\IPython\extensions\autoreload.py", line 397, in update_generic
    update(a, b)
  File "g:\Meine Ablage\NewsTrading\trading_bot\.venv\Lib\s

In [2]:
# Settings
target_col_name = MODEL_CONFIG.target_col_name
bert_model_name = MODEL_CONFIG.transformer_hugface_id
FROM_SCRATCH = True
batch_size = 2
epochs = 1
tokenizer = BertTokenizerFast.from_pretrained(bert_model_name)
loss_function = MODEL_CONFIG.loss_function
learning_rate = 0.01 # 5e-5 (slow) for bert, 0.3 (fast) for new feed forward

In [3]:
pt_version = torch.__version__
print(f"[INFO] Current PyTorch version: {pt_version} (should be 2.x+)")
torch.cuda.is_available()

[INFO] Current PyTorch version: 2.1.1+cu118 (should be 2.x+)


True

In [4]:
# Download dataset
dataset = pd.read_parquet(config.data.merged)
dataset.shape[0]

419094

In [6]:
# Filter out Stocks... TODO: put this into filter interface and make configurable in model_config
dataset = dataset[
    (dataset["unadj_open"] >= 2) &          # penny stocks
    (dataset["dollar_volume"] >= 30_000) &  # illiquid stocks TODO: this has look-ahead bias
    (dataset["staleness"] <= 0.9)           # repeat news
                  ]
print(dataset.shape[0])
dataset.dropna(inplace=True)
print(dataset.shape[0])

254166
247271


C:\Users\Adria\AppData\Local\Temp\ipykernel_47176\183389681.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataset.dropna(inplace=True)


In [7]:
dataset: pd.DataFrame = MODEL_CONFIG.splitter.add_splits(dataset)

173089 samples in training set.
 49454 samples in validation set.
 24728 samples in testing set.


g:\Meine Ablage\NewsTrading\trading_bot\src\model\splits.py:47: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dat["split"] = "training"
g:\Meine Ablage\NewsTrading\trading_bot\src\model\splits.py:50: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dat["split"] = dat["split"].astype("category")


In [28]:
train_dataloader = get_data_loader_from_dataset(dataset=dataset,
                                                split="training",
                                                batch_size=batch_size,
                                                label_col=target_col_name,
                                                data_loader_kwargs=dict(shuffle=True,
                                                                        pin_memory=True))

dataset.index.name=None


In [21]:
validation_dataloader = get_data_loader_from_dataset(dataset=dataset,
                                                split="validation",
                                                batch_size=batch_size,
                                                label_col=target_col_name,
                                                data_loader_kwargs=dict(shuffle=True,
                                                                        pin_memory=True))

dataset.index.name=None


In [26]:
model: nn.Module = MODEL_CONFIG.neural_net(bert_model_name=MODEL_CONFIG.transformer_hugface_id, 
                                            num_classes=3)

if MODEL_CONFIG.input_params_path:
    model.load_state_dict(torch.load(MODEL_CONFIG.input_params_path))
    
MyBertModule.deactivate_learning_for_layer(model.bert)

# .compile currently isn't supported for Windows
# model = torch.compile(model)

if __name__ == "__main__":
    torch.cuda.empty_cache()

    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    print(f"{device=}")
    model.to(device)

    # Optimizer, scheduler and loss function
    optimizer = AdamW(model.parameters(), lr=learning_rate, eps=1e-8)

    total_steps = len(train_dataloader) * epochs
    scheduler = get_linear_schedule_with_warmup(optimizer,
                                                num_warmup_steps=0,
                                                num_training_steps=total_steps)

    # Training
    model, training_stats = train(model,
                                  optimizer,
                                  scheduler,
                                  loss_function,
                                  epochs,
                                  train_dataloader,
                                  validation_dataloader,
                                  device,
                                  clip_value=2)

    df_stats = pd.DataFrame(data=training_stats)
    print(df_stats)

    # Store Model
    torch.save(model.state_dict(), MODEL_CONFIG.output_params_path)

device=device(type='cuda')


KeyboardInterrupt: 

In [ ]:
if google_colab:
    from google.colab import runtime
    runtime.unassign()